# MRA Quality Assurance Notebook 

Use this notebook to investigate a single layer or query

Follow the steps below

1. Select the layer and optional filter
2. Login to ArcGIS
3. Execute Test

You can execute each step seperately or all-at-once using run-all

## Select the Layer and Optional SQL filter

Use comments ("#") to select amongst the options or define your own

In [ ]:
LAYER = "ceea424c2f2149c38f1cb3be46653325" # MRA Main
FILTER = "1=1" # No Filter

## Login to ArcGIS

In [ ]:
from arcgis.gis import GIS

gis = GIS("https://msar.maps.arcgis.com", "MRA_Admin") 

# Execute Tests

In [ ]:
from mratools import check_incidents, check_points

In [ ]:
feature_layer = gis.content.get(LAYER)

In [ ]:
incidents = feature_layer.layers[0].query(where=FILTER) #Feature Set

In [ ]:
points_found = feature_layer.layers[1].query(where=FILTER) #Feature Set

In [ ]:
results_i = [_ for _ in check_incidents(incidents)]
results_p = [_ for _ in check_points(points_found)]
print ("Incident Test Results: ", results_i)
print ("Points Test Results: ", results_p)

## Calculate Statistics

In [ ]:
uuids = []
teams = {}
geometries = {}
dates = {}
versions = {}

for incident in incidents:
    uuid = incident.get_value("Incident_UUID")
    team = incident.get_value("Team")
    date = incident.get_value("Incident_Date") #in millisecond unix time
    version = incident.get_value("Form_Version")
    
    _geometry = incident.geometry 
    if _geometry is not None:
        geometry = (_geometry['x'], _geometry['y'],)
    else:
        geometry = None
    
    if team not in teams:
        teams[team] = []
    
    if geometry not in geometries:
        geometries[geometry] = []
        
    if date not in dates:
        dates[date] = []
        
    if version not in versions:
        versions[version] = []
        
    teams.get(team).append(uuid)
    geometries.get(geometry).append(uuid)
    dates.get(date).append(uuid) 
    versions.get(version).append(uuid)
    uuids.append(uuid)
    

# Results

In [ ]:
PRINT_URL = True
FORMAT_URL="https://msar.maps.arcgis.com/apps/webappviewer/index.html?id=2c16b6569d174d20a278e683f289658e&query=MRA%20Mission%20Data%20-%20653325%20(Current%20Year),Incident_UUID,"

## Null Incident ID

In [ ]:
print("Null Incident UUIDs:", None in uuids)

## Duplicate Incident ID

In [ ]:
duplicates = {}
for uuid in uuids:
    if uuids.count(uuid)>1:
        duplicates[uuid] = duplicates.get(uuid,0) + 1

for k,v in duplicates.items():
    print ("Duplicate:", k, "Count:", v)
    if PRINT_URL: print(FORMAT_URL+k) 
    
print("Total number of Duplicate Incidents", sum(duplicates.values()))
print("Unique number of Duplicate Incident_UUID's", len(duplicates))

## Erroneous Geometries

In [ ]:
n = 0
for k,v in geometries.items():
    if k is None or k [0] < -170 or k[0] > -20 or k[1] <5 or k[1] > 75 :
        print ("Point:", k, "Incidents:", *v)#, "\n")
        if PRINT_URL: print (*[FORMAT_URL+_ for _ in v])
        n = n+len(v)
print("Total Number of Erroneous Geometries:", n)

## Erroneous Dates

In [ ]:
import datetime

#for k,v in dates.items():
#    print (k)
#    print (k/1000)
#    try:
#        print(datetime.datetime.fromtimestamp(k/1000))
#    except:
#        print("error")

datetimes = {datetime.datetime.fromtimestamp(k/1000):v for k,v in dates.items()}
datetimes = {k:datetimes[k] for k in sorted(datetimes,reverse = True)} 

n = 0
for k,v in datetimes.items():
    if k < datetime.datetime(2013,1,1) or k > datetime.datetime.now():
        print("Date:", k, "Incidents:", *v)#, "\n") 
        if PRINT_URL: print (*[FORMAT_URL+_ for _ in v])
        n = n+len(v)
print("Total Erroneous Dates: ", n)
#error_dates = {k:v for k,v in datetimes.items() if k < datetime.datetime(2013,1,1) or k > datetime.datetime.now()}

## Teams

In [ ]:
t_v = {k:len(v) for k,v in teams.items()}
sorted_teams = sorted(t_v, key=t_v.get, reverse=True)
for team in sorted_teams:
    print("Team:", team, len(teams[team]))

## Form Version

In [ ]:
v_v = {k:len(v) for k,v in versions.items()}
sorted_versions = sorted(v_v, key=v_v.get, reverse=True)
for version in sorted_versions:
    print("Version:", version, len(versions[version]))